In [18]:
from paddleocr import PaddleOCR, draw_ocr
import os
import pickle 
from PIL import Image
import pandas as pd
import numpy as np
import math
import jieba
from sklearn.utils import Bunch
from sklearn.feature_extraction.text import TfidfTransformer  # TF-IDF向量转换类
from sklearn.feature_extraction.text import TfidfVectorizer  # TF_IDF向量生成类
from sklearn.naive_bayes import MultinomialNB  # 多项式贝叶斯算法
import random
import re

In [10]:
def saveFile(path, result):
    with open(path, 'a', errors='ignore') as file:
        file.write(result)
def readFile(path):
    with open(path, 'r', errors='ignore') as file:  
        content = file.read()
        return content

In [16]:
#统计数字数量
def count_digits(s):
    return sum(1 for char in s if char.isnumeric())/len(s)
#只保留数字
def extract_digits(s):
    return re.findall(r'\d+', s)

In [12]:
# 获取距离
def distance(p1,p2):
    ans=[]
    for i in range(4):
        for j in range(4):
            an=math.sqrt((p1[i][0]-p2[j][0])**2+(p1[i][1]-p2[j][1])**2)
            ans.append(an)
    return np.array(ans).min()/500


In [13]:
def get_left_distance(p1,p2):
    an=math.sqrt((p1[1][0]-p2[0][0])**2+16*(p1[1][1]-p2[0][1])**2)
    return an/500

# 训练文本分类模型
## 准备数据
### jieba分词

In [11]:
#公司名称
company_name=pd.read_csv("company_name.txt",sep='\t')
cutPath="C:\daima\ocr\cut_company_name.txt"
with open(cutPath, 'w') as file:
    file.write(" ")
for i in company_name['编号']:
    cutResult=" ".join(jieba.cut(i))+"\n"
    saveFile(cutPath , cutResult)

Building prefix dict from the default dictionary ...
[2024-12-18 23:34:47,509] [   DEBUG] __init__.py:113 - Building prefix dict from the default dictionary ...


Loading model from cache C:\Users\86151\AppData\Local\Temp\jieba.cache
[2024-12-18 23:34:47,514] [   DEBUG] __init__.py:132 - Loading model from cache C:\Users\86151\AppData\Local\Temp\jieba.cache
Loading model cost 0.925 seconds.
[2024-12-18 23:34:48,438] [   DEBUG] __init__.py:164 - Loading model cost 0.925 seconds.
Prefix dict has been built successfully.
[2024-12-18 23:34:48,454] [   DEBUG] __init__.py:166 - Prefix dict has been built successfully.


In [12]:
#人名
people_name=pd.read_csv("Chinese_Names_Corpus_Gender.txt")
p_cutPath="C:\daima\ocr\cut_people_name.txt"
with open(p_cutPath, 'w') as file:
    file.write(" ")
count=0
for i in range(520):
    j=random.randint(1,100000)
    cutResult=" ".join(jieba.cut(people_name['dict'][j]))+"\n"
    count+=1
    saveFile(p_cutPath , cutResult)

### 装入banch

In [13]:
aimfile = "C:\\daima\\ocr\\aimfile.dat"
bunch = Bunch(label=[], index=[], contents=[])
count=0
with open (p_cutPath, 'r') as file:
    line = file.readline();
    while line :
        bunch.contents.append(line)
        bunch.label.append("p")
        bunch.index.append(count)
        line = file.readline();count+=1
with open(cutPath,'r') as file :
    line = file.readline();
    while line :
        bunch.contents.append(line)
        bunch.label.append("c")
        bunch.index.append(count)
        line = file.readline();count+=1
with open(aimfile, 'wb') as file_obj:
    pickle.dump(bunch,file_obj)
        

In [14]:
def readBunch(path):
    with open(path, 'rb') as file:
        bunch = pickle.load(file)
        #pickle.load(file)
        #函数的功能：将file中的对象序列化读出。
    return bunch
def writeBunch(path, bunchFile):
    with open(path, 'wb') as file:
        pickle.dump(bunchFile, file)
def getStopWord(inputFile):
    stopWordList = readFile(inputFile).splitlines()
    return stopWordList
 

### 求得TF-IDF向量

In [15]:
TFPath="C:\\daima\\ocr\\TFfile.dat"
bunch=readBunch(aimfile)
tfidfspace = Bunch(label=bunch.label, index=bunch.index, tdm=[],vocabulary={})
    # 初始化向量空间
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5)
transformer = TfidfTransformer()  # 该类会统计每个词语的TF-IDF权值
# 文本转化为词频矩阵，单独保存字典文件
tfidfspace.tdm = vectorizer.fit_transform(bunch.contents)
tfidfspace.vocabulary = vectorizer.vocabulary_   #获取词汇
writeBunch(TFPath, tfidfspace)

## 将数据转为banch

In [16]:
def to_banch(test_aimfile,results):
    test_bunch = Bunch(label=[], index=[], contents=[])
    count=0
    cut_str = []
    for line in results[0]:
        cutResult=jieba.cut(line[1][0])
        test_bunch.contents.append(" ".join(cutResult))
        test_bunch.index.append(count)
        count+=1
    with open(test_aimfile, 'wb') as file_obj:
        pickle.dump(test_bunch,file_obj)

In [41]:
results=get_picture("picture1.jpg")

[2024/12/22 00:50:16] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\86151/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\86151/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

In [33]:
results[0][22]

[[[331.0, 1319.0], [389.0, 1319.0], [389.0, 1720.0], [331.0, 1720.0]],
 ('东莞股份有限公司印制', 0.8228614926338196)]

In [45]:
all_line=""
for line in results[0]:
    all_line+=line[1][0]+", "

In [46]:
all_line

'安微省医疗门诊收费票据, 票据号：0025696429, 门诊号：2010100229, 好政部监, (19）, 皮山县人民医院, 00188, 医疗机构类型：, 医疗机构名称：, 自费医疗, 王宇峰, ., 社会保隆号码, 医保类, 姓名：, S9, 金额, 项目, 项目, 120, 第一装, 收据联, 东莞股份有限公司印制, 盖章有效遗失不补, 2020-07-1718:39:43, 就诊时间：, 119-00, 贰佰陆拾圆整, (）, 合计（大写）：, 120, 18:39-43, 医保统筹支, sts, 收款人（签章）：, 年, 月, 教款单位, 章）, 日, '

In [47]:
messages=[
        {"role": "user", 
         "content": f"下面有一些字符串 {all_line} 将其中的属性和属性名配对。\n"}]
re=API.get_respone_from_api(1,messages)
re

任务2完成，耗时: 0.92s


"Based on the provided text, I've identified the attributes and their corresponding attribute names. Here's the pairing:\n\n1. **票据号** (Ticket Number): 0025696429\n2. **门诊号** (Clinic Number): 2010100229\n3. **医疗机构类型** (Medical Institution Type): (blank)\n4. **医疗机构名称** (Medical Institution Name): 皮山县人民医院 (Pishan County People's Hospital)\n5. **社会保隆号码** (Social Insurance Number): (blank)\n6. **医保类** (Medical Insurance Type): (blank)\n7. **姓名** (Name): 王宇峰 (Wang Yufeng)\n8. **金额** (Amount): 120\n9. **项目** (Project): 第一装 (First Aid)\n10. **收据联** (Receipt Number): (blank)\n11. **就诊时间** (Treatment Time): ("

In [ ]:
test_aimfile = "C:\\daima\\ocr\\test_set_file.dat"
test_bunch = Bunch(label=[], index=[], contents=[])
count=0
cut_str = []
for line in results[0]:
    cutResult=jieba.cut(line[1][0])
    test_bunch.contents.append(" ".join(cutResult))
    test_bunch.index.append(count)
    count+=1
with open(test_aimfile, 'wb') as file_obj:
    pickle.dump(test_bunch,file_obj)
    

In [23]:
def getTestSpace(testSetPath, trainSpacePath, testSpacePath):
    bunch = readBunch(testSetPath)
    # 构建测试集TF-IDF向量空间
    testSpace = Bunch(label=bunch.label, filenames=bunch.index, tdm=[],
                      vocabulary={})
    # 导入训练集的词袋
    trainbunch = readBunch(trainSpacePath)
    # 使用TfidfVectorizer初始化向量空间模型  使用训练集词袋向量
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 vocabulary=trainbunch.vocabulary)
    transformer = TfidfTransformer()
    testSpace.tdm = vectorizer.fit_transform(bunch.contents)
    testSpace.vocabulary = trainbunch.vocabulary
    # 持久化
    writeBunch(testSpacePath, testSpace)

In [24]:
def bayesAlgorithm(trainPath, testPath):
    trainSet = readBunch(trainPath)
    testSet = readBunch(testPath)
    clf = MultinomialNB(alpha=0.001).fit(trainSet.tdm, trainSet.label)
    predicted = clf.predict(testSet.tdm)
    return predicted

In [25]:
testSpacePath="C:\\daima\\ocr\\test_space_file.dat"
getTestSpace(test_aimfile, TFPath, testSpacePath)
bayesAlgorithm(TFPath, testSpacePath)

array(['c', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'c', 'p', 'p', 'p',
       'p', 'p', 'p', 'p', 'p', 'p', 'p', 'c', 'p', 'c', 'p', 'p', 'p',
       'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',
       'p'], dtype='<U1')

## 利用OCR技术获得发票图片上的数据

In [14]:
def get_picture(image_path):
    # 初始化 OCR 模型
    ocr = PaddleOCR(use_angle_cls=True, lang='ch')  # 支持中文
    # 文字检测和识别
    results = ocr.ocr(image_path, cls=True)
    return results

## 获取指定路径指定属性的数据

In [7]:
def get_ans(results):
    count=0;p=[];m=[];j=[];t=[];c=[]
    for line in results[0]:
        #print(f"识别文字: {line[1][0]}, 置信度: {line[1][1]}")
        index_c=line[1][0].find("机构名称")
        index_p=line[1][0].find("票据号")
        index_m=line[1][0].find("门诊号")
        index_t=line[1][0].find("时间")
        index_j=line[1][0].find("金额")
        if index_c>-1:
            #print(f"包含机构的字符串：{line[1][0]}")
            c.append(count)
        if index_p>-1:
            #print(f"包含票据号的字符串：{line[1][0]}")
            p.append(count)
        if index_m>-1:
            #print(f"包含门诊号的字符串：{line[1][0]}")
            m.append(count)
        if index_t>-1:
            #print(f"包含时间的字符串：{line[1][0]}")
            t.append(count)
        if index_j>-1:
            #print(f"包含金额的字符串：{line[1][0]}")
            j.append(count)
        count+=1
    mp=[];mm=[];mj=[];mt=[];mc=[]
    for line in results[0]:
        d_p=distance(line[0],results[0][p[0]][0])
        d_m=distance(line[0],results[0][m[0]][0])
        d_j=distance(line[0],results[0][j[0]][0])
        d_t=distance(line[0],results[0][t[0]][0])
        d_c=get_left_distance(results[0][c[0]][0],line[0])
        cd=count_digits(line[1][0])
        mp.append(cd-d_p)
        mm.append(cd-d_m)
        mj.append(cd-d_j)
        mt.append(cd-d_t)
        mc.append(-d_c)
    i_p=np.argmax(np.array(mp))
    i_m=np.argmax(np.array(mm))
    i_j=np.argmax(np.array(mj))
    i_t=np.argmax(np.array(mt))
    i_c=np.argmax(np.array(mc))
    print(i_p,i_m,i_j,i_t,i_c)
    digits_m = extract_digits(results[0][i_m][1][0])
    digits_p = extract_digits(results[0][i_p][1][0])
    digits_j = extract_digits(results[0][i_j][1][0])
    #print(f"{digits_p[0]},{digits_m[0]},{digits_j[0]},"+results[0][i_t][1][0]+results[0][i_c][1][0])
    
    return [results[0][i_c][1][0],digits_p[0],digits_m[0],results[0][i_t][1][0],digits_j[0]]
data = pd.DataFrame(columns=["公司","票据号","门诊号","时间","金额"])

In [8]:
def get_data(path):
    results=get_picture(path)
    return get_ans(results)

In [15]:
def half_auto_get_ans(por,results):
    length=len(por)
    count=0;answers=np.zeros(length);ans=[]
    for line in results[0]:
        count+=1
        for i in range(length):
            index=line[1][0].find(por[i][0])
            if index>-1:
                answers[i]=count
    maxs=np.full(length, -100.0);indexs=np.zeros(length);count=0
    for line in results[0]:
        count+=1
        for i in range(length):
            if answers[i]==0:
                continue
            elif por[i][1]=="i":
                d=distance(line[0],results[0][int(answers[i]-1)][0])
                cd=count_digits(line[1][0])
                if cd-d>maxs[i]:
                    indexs[i]=count
                    maxs[i]=cd-d
            elif por[i][1]=="c":
                d=-get_left_distance(results[0][int(answers[i]-1)][0],line[0])
                if d>maxs[i]:
                    indexs[i]=count
                    maxs[i]=d
    for i in range(length):
        if answers[i]==0:
            ans.append("#Error#")
        elif por[i][1]=="i":
            ans.append(extract_digits(results[0][int(indexs[i]-1)][1][0])[0])
        elif por[i][1]=="c":
            ans.append(results[0][int(indexs[i]-1)][1][0])
    return ans

In [16]:
def half_auto_get_data(path,por,type,aimpath):
    col=[]
    for i in por :
        col.append(i[0])
    data = pd.DataFrame(columns=col)
    if type=="f":
        data.loc[len(data)]=half_auto_get_ans(por,get_picture(path))
    elif type=="d":
        fatherLists = os.listdir(path)
        for c_path in fatherLists:
            child_path=path+c_path
            data.loc[len(data)]=half_auto_get_ans(por,get_picture(child_path))
    data.to_csv(aimpath)
    return data

### 获取数据类型列表

In [17]:
import API

In [18]:
def get_por_api(por_path):
    data=pd.read_csv(por_path)
    str=data.columns
    results = []
    all=""
    for i in str:
        all=all+i+" "
    messages=[
        {"role": "user", 
         "content": f"请分辨 {all} 属于哪种数据类型，其中数据类型包括两类：数字串（用i表示） 或者字符串（用s表示）."
         +"请输出一个序列，每项都为i或者，表示各个词的类，除了这个序列不要输出其他东西。\n"}]
    re=API.get_respone_from_api(1,messages)
    print(i+"type is "+re)
    results.append(re)
    return results
por_path="aim.csv"
por=get_por_api(por_path)
por

任务2完成，耗时: 3.40s
公司名称type is Please provide the data, and I'll categorize each item as either an integer (i) or a string (s).

Please go ahead and provide the data.


["Please provide the data, and I'll categorize each item as either an integer (i) or a string (s).\n\nPlease go ahead and provide the data."]

In [19]:
# 设置属性名及属性类型
por=[["门诊号","i"],["机构名称","c"]]

In [37]:
half_auto_get_data("./data1/",por,"d","data1_result.csv")

[2024/12/22 15:37:00] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\86151/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\86151/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

KeyboardInterrupt: 

In [ ]:
half_auto_get_data('picture1.jpg',por,"f","picture1_result.csv")

In [5]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import time

In [7]:
def ui():
    
    # 创建主窗口
    root = tk.Tk()
    root.title("简单的 UI 示例")
    root.geometry("300x400")

    # 标签
    label1 = tk.Label(root, text="请输入读入地址:")
    label1.pack()
    entry1 = tk.Entry(root)
    entry1.pack()
    label2 = tk.Label(root, text="请输入读入地址类型:")
    label2.pack()
    entry2 = tk.Entry(root)
    entry2.pack()
    label3 = tk.Label(root, text="请输入输出地址:")
    label3.pack()
    entry3 = tk.Entry(root)
    entry3.pack()

    #    按钮点击事件
    def greet():
        in_path = entry1.get()
        in_type = entry2.get()
        out_path = entry3.get()
        half_auto_get_data(in_path,por,in_type,out_path)
        messagebox.showinfo("问候")

    # 按钮
    button = tk.Button(root, text="提交", command=greet)
    button.pack(pady=10)

    # 运行主循环
    root.mainloop()

In [ ]:
# 创建主窗口
root = tk.Tk()
root.title("进度条示例")
root.geometry("300x150")

# 创建进度条
progress = ttk.Progressbar(root, orient=tk.HORIZONTAL, length=200, mode='determinate')
progress.pack(pady=20)

# 更新进度的函数
def start_progress():
    progress['value'] = 0  # 重置进度值
    root.update_idletasks()  # 更新界面

    for i in range(101):  # 从 0 到 100
        time.sleep(0.05)  # 模拟工作
        progress['value'] = i  # 更新进度值
        root.update_idletasks()  # 更新界面

# 按钮开始更新进度
start_button = tk.Button(root, text="开始", command=start_progress)
start_button.pack(pady=10)

# 运行主循环
root.mainloop()

In [8]:
import json
import random
from pathlib import Path
import requests
import time

# Groq API 配置
api_url = "https://api.groq.com/openai/v1/chat/completions"
api_key = "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"  # 请确保你的API密钥正确
headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

def get_respone_from_api(idx,message):
    payload = {
        "model": "llama3-8b-8192",
        "messages": message,
        "max_tokens": 200,
        "temperature": 0.7
    }
    while True:  # 无限循环，直到任务成功或不再触发速率限制
        start_time = time.time()
        try:
            response = requests.post(api_url, headers=headers, json=payload)
            end_time = time.time()

            if response.status_code == 200:
                result = response.json()
                model_response = result["choices"][0]["message"]["content"].strip()
                print(f"任务{idx + 1}完成，耗时: {end_time - start_time:.2f}s")
                return model_response  # 任务成功，退出当前循环
            elif response.status_code == 429:  # 触发速率限制
                wait_time = float(response.json().get("error", {}).get("message", "").split("in ")[-1].split("ms")[0]) / 1000
                print(f"任务 {idx + 1} 速率限制，等待 {wait_time:.2f}s 重试...")
                time.sleep(wait_time)  # 按照提示的时间等待
            else:
                print(f"任务 {idx + 1} 出错，状态码: {response.status_code}, 响应: {response.text}")
                break  # 其他错误不重试
        except Exception as e:
            print(f"任务 {idx + 1} 出错，网络错误: {str(e)}")
            break
    return 0

In [20]:
results = get_picture("picture1.jpg")

[2024/12/22 14:38:00] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\86151/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\86151/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

In [27]:
results[0][0][1][0]

'安微省医疗门诊收费票据'

In [21]:
s=" "
for line in results[0]:
    s+=(line[1][0]+" ")

In [29]:
s

' 安微省医疗门诊收费票据 票据号：0025696429 门诊号：2010100229 好政部监 (19） 皮山县人民医院 00188 医疗机构类型： 医疗机构名称： 自费医疗 王宇峰 . 社会保隆号码 医保类 姓名： S9 金额 项目 项目 120 第一装 收据联 东莞股份有限公司印制 盖章有效遗失不补 2020-07-1718:39:43 就诊时间： 119-00 贰佰陆拾圆整 (） 合计（大写）： 120 18:39-43 医保统筹支 sts 收款人（签章）： 年 月 教款单位 章） 日 '

## api


In [27]:
import json
import random
from pathlib import Path
import requests
import time

In [ ]:
# Groq API 配置
api_url = "https://api.groq.com/openai/v1/chat/completions"
api_key = "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"  # 请确保你的API密钥正确
headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

In [28]:
def get_respone_from_api(idx,message):
    payload = {
        "model": "llama3-8b-8192",
        "messages": message,
        "max_tokens": 200,
        "temperature": 0.7
    }
    while True:  # 无限循环，直到任务成功或不再触发速率限制
        start_time = time.time()
        try:
            response = requests.post(api_url, headers=headers, json=payload)
            end_time = time.time()

            if response.status_code == 200:
                result = response.json()
                model_response = result["choices"][0]["message"]["content"].strip()
                print(f"任务{idx + 1}完成，耗时: {end_time - start_time:.2f}s")
                return model_response  # 任务成功，退出当前循环
            elif response.status_code == 429:  # 触发速率限制
                wait_time = float(response.json().get("error", {}).get("message", "").split("in ")[-1].split("ms")[0]) / 1000
                print(f"任务 {idx + 1} 速率限制，等待 {wait_time:.2f}s 重试...")
                time.sleep(wait_time)  # 按照提示的时间等待
            else:
                print(f"任务 {idx + 1} 出错，状态码: {response.status_code}, 响应: {response.text}")
                break  # 其他错误不重试
        except Exception as e:
            print(f"任务 {idx + 1} 出错，网络错误: {str(e)}")
            break
    return 0

In [23]:
message = [{"role": "user", "content": "请帮我识别以下用空格分隔的字符串，找出可能是数字的部分，并返回这些数字的个数和位置。格式为index:第几个字符串 number:识别的数字 len :数字长度 \n"+s}]

In [34]:
api_respone=get_respone_from_api(0,message)

任务1完成，耗时: 0.78s


In [35]:
test=api_respone.split("\n")
print(test)


["I'd be happy to help you identify the possible numbers in the given string and provide their count, position, and length.", '', 'Here are the results:', '', '1. index: 2, number: 0025696429, len: 10', '2. index: 6, number: 2010100229, len: 9', '3. index: 14, number: 19, len: 2', '4. index: 17, number: 00188, len: 5', '5. index: 31, number: 2020, len: 4', '6. index: 34, number: 1718, len: 4', '7. index: 38, number: 119, len: 3', '8. index: 41, number: 120, len: 3', '9. index: 43, number: 18, len: 2', '10. index: 51, number']


In [1]:
import cv2
import numpy as np

In [9]:
def sharpen_image(image_path,s_dic):
    # 读取图片
    image = cv2.imread(image_path)
    
    # 创建锐化内核
    sharpening_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])

    # 应用锐化滤波器
    sharpened_image = cv2.filter2D(image, -1, sharpening_kernel)

    # 保存锐化后的图片
    cv2.imwrite('sharpened/'+image_path, sharpened_image)

In [10]:
sharpen_image("picture1.jpg","sharpened/")

In [14]:
image_path="picture1.jpg"
image = cv2.imread(image_path)
    
    # 创建锐化内核
sharpening_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])

    # 应用锐化滤波器
sharpened_image = cv2.filter2D(image, -1, sharpening_kernel)

    # 保存锐化后的图片
cv2.imwrite('s//'+image_path, sharpened_image)

False

## evaluation

In [5]:
import json
import pandas as pd

In [3]:
# 指定 Java 文件的路径
json_file_path = 'label.json'

# 读取 Java 文件
with open(json_file_path, 'r', encoding='utf-8') as file:
    content = json.load(file)  # 读取文件内容
    print(content)  # 打印文件内容


[{'filename': '47.jpg', 'entities': [{'text': '杨性坤', 'label': '购买方名称'}, {'text': '纯电动多用途乘用车', 'label': '车辆类型'}, {'text': '本田牌MZLFOCLGRPOU', 'label': '厂牌型号'}, {'text': '上海', 'label': '产地'}, {'text': 'TG321316003581', 'label': '发动机号'}, {'text': '浙江宝马汽车销售服务有限公司', 'label': '销售单位名称'}, {'text': '867186208750413331', 'label': '纳税人识别号'}, {'text': '花旗银行（中国）有限公司杭州分行', 'label': '开户银行'}, {'text': '73803.84', 'label': '增值税税额'}, {'text': '244247.79', 'label': '不含税价'}, {'text': '国家税务总局杭州市萧山区税务局经济技术开发区税务所13301099200', 'label': '主管税务机关及代码'}]}, {'filename': '34.jpg', 'entities': [{'text': '丁灵琳', 'label': '购买方名称'}, {'text': '纯电动多用途乘用车', 'label': '车辆类型'}, {'text': '红旗牌8RPNG1SLSL', 'label': '厂牌型号'}, {'text': '上海', 'label': '产地'}, {'text': 'TG321316003581', 'label': '发动机号'}, {'text': '北京长征销售服务有限公司', 'label': '销售单位名称'}, {'text': '581978217200132381', 'label': '纳税人识别号'}, {'text': '花旗银行（中国）有限公司杭州分行', 'label': '开户银行'}, {'text': '261182.45', 'label': '增值税税额'}, {'text': '244247.79', 'label': '不含税价'}, {'text': '国家

In [13]:
po=["发票代码","购买方名称"]

In [15]:
data=pd.read_csv("test_result.csv")
data['发票代码'][1]

123084821864

In [25]:
len(data)
correct=0
for line in content:
    for por in line['entities']:
        if por['label'] in po:
            data_name=data[por['label']][int(extract_digits(line['filename'])[0])-30]
            #print(por['text'],por['label'],data_name)
            if por['text']==data_name:
                correct+=1
print(correct/data.__len__())

0.9666666666666667
